### Collabrative Filtering/Recommender Systems

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [4]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [6]:
data = spark.read.csv('movielens_rating.csv',inferSchema=True,header=True)

In [7]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [9]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [10]:
train,test = data.randomSplit([0.8,0.2])

In [12]:
als = ALS(maxIter=5, regParam = 0.01, userCol='userId',itemCol='movieId',ratingCol='rating')

In [14]:
model = als.fit(train)

In [15]:
predictions = model.transform(test)

In [16]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    13|   2.226652|
|     31|   1.0|     5|  1.5493863|
|     31|   1.0|    24|  1.4642148|
|     31|   1.0|    29|  1.5896568|
|     31|   3.0|    14|  1.8858713|
|     85|   1.0|    28|  2.8691587|
|     85|   5.0|    16|  2.5956595|
|     85|   1.0|    25|  4.2865067|
|     85|   1.0|     2| -1.3962071|
|     65|   1.0|    16|-0.06871903|
|     65|   1.0|    19| -0.7703574|
|     53|   1.0|     6|   1.558691|
|     53|   1.0|    23| 0.41489506|
|     78|   1.0|    28|  1.1525794|
|     78|   1.0|     1|   0.932937|
|     78|   1.0|    17|  1.1831447|
|     78|   1.0|    24| 0.37759697|
|     34|   3.0|     3|-0.20758468|
|     34|   1.0|    19|   0.667323|
|     34|   1.0|     4|   1.362011|
+-------+------+------+-----------+
only showing top 20 rows



In [17]:
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [18]:
rmse = evaluator.evaluate(predictions)

In [19]:
print('RMSE')
print(rmse)

RMSE
1.7863109658233756


In [22]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [23]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     10|    11|
|     13|    11|
|     22|    11|
|     32|    11|
|     36|    11|
|     48|    11|
|     50|    11|
|     62|    11|
|     64|    11|
|     79|    11|
|     86|    11|
+-------+------+



In [24]:
recommendations = model.transform(single_user)

In [26]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     22|    11|  4.002632|
|     36|    11| 3.2782373|
|     62|    11| 1.9257803|
|     64|    11|  1.678016|
|     13|    11| 1.5212269|
|     10|    11| 1.1891404|
|     86|    11|  0.963177|
|     48|    11|0.52622426|
|     50|    11| 0.4878311|
|     79|    11|0.39733136|
|     32|    11|-1.3217897|
+-------+------+----------+

